In [4]:
from sage.crypto.boolean_function import BooleanFunction

Створюємо кільце поліномів

In [5]:
B = BooleanPolynomialRing(6, 'x', order = 'degrevlex')
B.inject_variables()

Defining x0, x1, x2, x3, x4, x5


Зчитуємо дані з файлів та переводимо їх у належний вигляд

In [6]:
f = open("VG9.txt", "r")
gamma=list(map(int, list(f.readlines()[0])))
f.close()

In [7]:
f = open("VCF9.txt", "r")
combining_function = f.readlines()[0][:-1]
f.close()
combining_function

'x_{0} x_{2} x_{4} + x_{1} x_{2} x_{4} + x_{1} x_{4} + x_{2} x_{3} x_{4} + x_{2} x_{4} x_{5} + x_{2} x_{4} + x_{2} + x_{3} x_{4'

In [8]:
combining_function=combining_function.replace("_{","")
combining_function=combining_function.replace("} x"," * x")
combining_function=combining_function.replace("} +"," +")
combining_function

'x0 * x2 * x4 + x1 * x2 * x4 + x1 * x4 + x2 * x3 * x4 + x2 * x4 * x5 + x2 * x4 + x2 + x3 * x4'

In [9]:
combining_function = eval(combining_function)
combining_function

x4*x2*x0 + x4*x2*x1 + x4*x3*x2 + x5*x4*x2 + x4*x1 + x4*x2 + x4*x3 + x2

In [10]:
BooleanFunction(combining_function)

Boolean function with 6 variables

Використовуємо перетворення Уолша-Адамара та будуємо статистичні аналоги

In [11]:
walsh_adamar_spectre = list(BooleanFunction(combining_function).walsh_hadamard_transform())
print(walsh_adamar_spectre)

[0, 0, 0, 0, 32, 0, 0, 0, 0, 0, 16, 0, 0, 0, 16, 0, 0, 0, 0, 0, 32, 0, 0, 0, 0, 0, -16, 0, 0, 0, -16, 0, 0, 16, 0, 0, 0, -16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -16, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
not_null_walsh = [i for i in range(len(walsh_adamar_spectre)) if walsh_adamar_spectre[i]!=0]
not_null_walsh

[4, 10, 14, 20, 26, 30, 33, 37, 49, 53]

In [13]:
analogs=[]

for i in range(len(not_null_walsh)):
    walsh_coeff = walsh_adamar_spectre[not_null_walsh[i]]
    binary_i = [(not_null_walsh[i] >> j) & 1 for j in range(6)]
    
    analog = ""
    for a in range(6):
        if binary_i[a] == 1:
            analog += f'x{a}'
            analog += ' + '
    analog=analog[:-2]
    if walsh_coeff < 0:
        analog += '+ 1'
    print("g" + str(not_null_walsh[i]) + " = " + str(analog))
        
    analogs.append(eval(analog))

g4 = x2 
g10 = x1 + x3 
g14 = x1 + x2 + x3 
g20 = x2 + x4 
g26 = x1 + x3 + x4 + 1
g30 = x1 + x2 + x3 + x4 + 1
g33 = x0 + x5 
g37 = x0 + x2 + x5 + 1
g49 = x0 + x4 + x5 + 1
g53 = x0 + x2 + x4 + x5 


In [14]:
analogs

[x2,
 x1 + x3,
 x1 + x2 + x3,
 x2 + x4,
 x1 + x3 + x4 + 1,
 x1 + x2 + x3 + x4 + 1,
 x0 + x5,
 x0 + x2 + x5 + 1,
 x0 + x4 + x5 + 1,
 x0 + x2 + x4 + x5]

In [15]:
probabilities=[0 for i in range(len(not_null_walsh))]

In [16]:
for i in range(len(not_null_walsh)):
    for x in range(2**6):
        binary_x = [(x >> j) & 1 for j in range(6)]
        if combining_function(*binary_x) == analogs[i](*binary_x):
            probabilities[i] += 1
    probabilities[i]/=64

In [17]:
probabilities

[3/4, 5/8, 5/8, 3/4, 5/8, 5/8, 5/8, 5/8, 5/8, 5/8]

In [18]:
walsh=[walsh_adamar_spectre[not_null_walsh[i]] for i in range(len(not_null_walsh))]
walsh

[32, 16, 16, 32, -16, -16, 16, -16, -16, 16]

Обираємо 3 аналоги з якими будемо працювати та рахуємо для них значення Т

In [19]:
g1 = analogs[0]
g4 = analogs[3]
g6 = analogs[5]


epsilon=0.5
delta = 0.01
m = 10       #deg p = 10 = m
T1 = int(8*epsilon**(-2)*math.log(2**(m)/delta))     #k=1
T2 = int(8*epsilon**(-2)*math.log(2**(m*2)/delta))   #k=2
T4 = int(8*epsilon**(-2)*math.log(2**(m*4)/delta))   #k=4

print("T1 = " + str(T1))
print("T2 = " + str(T2))
print("T4 = " + str(T4))

T1 = 369
T2 = 590
T4 = 1034


In [20]:
%%cython

from libc . stdlib cimport malloc , free
cdef int LENGTH = 10

def gen_sequence (int poly, int init_state, int nbits):
    cdef int curr_state = init_state, s, i, j
    cdef int *res_array = <int *> malloc(sizeof(int)*nbits)
    try:
        for i in range(nbits):
            res_array[i] = (curr_state >> (LENGTH - 1)) & 1
            s = 0
            for j in range(LENGTH):
                s = s ^ (((curr_state & poly) >> j) & 1)
            curr_state = (curr_state << 1) | s
        return [x for x in res_array [:nbits]]
    finally:
        free(res_array)

In [21]:
def distance(seq1, seq2):
    return sum(abs(int(seq1[i]) - int(seq2[i])) for i in range(len(seq1)))

In [22]:
R.<x> = GF(2^11, 'a')[]  
polynoms = [x^10 + x^3 + 1, x^10 + x^7 + 1, x^10 + x^4 + x^3 + x^1 + 1, x^10 + x^8 + x^3 + x^2 + 1, 
            x^10 + x^8 + x^4 + x^3 + 1, x^10 + x^8 + x^5 + x^1 + 1]
polynoms_int = [int(''.join ([str(i) for i in p.list()[:-1]]), 2) for p in polynoms]

g1 = x2

In [23]:
c=2**10

In [24]:
%%time
states = {}
for i in range(c):
    sequence_x2 = gen_sequence(polynoms_int[2], i, T1)
    states[i] = distance(sequence_x2, gamma[:T1])
    
min_dist = min(states.values())
print("minimal distance = " + str(min_dist) + "  (" + str(min_dist/T1) + ")")
p2_states=[i for i in states if states[i] == min_dist]
print("possible initial state for p2 = " + str(p2_states))

minimal distance = 98  (0.26558265582655827)
possible initial state for p2 = [309]
CPU times: user 49.4 ms, sys: 0 ns, total: 49.4 ms
Wall time: 48.8 ms


g2 = x2 + x4

In [25]:
%%time
states = {}
sequence_x2 = gen_sequence(polynoms_int[2], 309, T2)
for i in range(c):
    sequence_x4 = gen_sequence(polynoms_int[4], i, T2)
    seq = [g4(0, 0, sequence_x2[j], 0, sequence_x4[j], 0) for j in range(T2)]
    states[i] = distance(seq, gamma[:T2])
    
min_dist = min(states.values())
print("minimal distance = " + str(min_dist) + "  (" + str(min_dist/T2) + ")")
states=[i for i in states if states[i] == min_dist]
print("possible initial state for p4 = " + str(states) + "  if initial state for p2 = " + str(309))

minimal distance = 143  (0.2423728813559322)
possible initial state for p4 = [911]  if initial state for p2 = 309
CPU times: user 3.91 s, sys: 0 ns, total: 3.91 s
Wall time: 3.91 s


g6 = x1 + x2 + x3 + x4 + 1

In [26]:
states = {}
sequence_x2 = gen_sequence(polynoms_int[2], 309, T4)
sequence_x4 = gen_sequence(polynoms_int[4], 911, T4)
for i in range(c):
    sequence_x1 = gen_sequence(polynoms_int[1], i, T4)
    for j in range(c):
        sequence_x3 = gen_sequence(polynoms_int[3], j, T4)
        seq = [g6(0, sequence_x1[k], sequence_x2[k], sequence_x3[k], sequence_x4[k], 0) for k in range(T4)]
        states[(i,j)] = distance(seq, gamma[:T4])

In [27]:
min_dist = min(states.values())
print("minimal distance = " + str(min_dist) + "  (" + str(min_dist/T4) + ")")
states=[(i, j) for (i, j) in states if states[(i, j)] == min_dist]
print("possible initial state for p1, p3 = " + str(states)+ "  if initial state for p2, p4 = " + str(309) + ", " + str(911))

minimal distance = 372  (0.3597678916827853)
possible initial state for p1, p3 = [(800, 470)]  if initial state for p2, p4 = 309, 911


In [28]:
l=500
sequence_x1 = gen_sequence(polynoms_int[1], 800, l)
sequence_x2 = gen_sequence(polynoms_int[2], 309, l)
sequence_x4 = gen_sequence(polynoms_int[4], 911, l)
sequence_x3 = gen_sequence(polynoms_int[3], 470, l)
flag = False
for i in range(c):
    if flag:
        break
    sequence_x0 = gen_sequence(polynoms_int[0], i, l)
    for j in range(c):
        sequence_x5 = gen_sequence(polynoms_int[5], j, l)
        seq = [combining_function(sequence_x0[k], sequence_x1[k], sequence_x2[k], sequence_x3[k], sequence_x4[k], sequence_x5[k]) for k in range(l)]
        if seq == gamma[:l]:
            print("possible initial state for p0, p5 = " + str(i) + ", " + str(j))
            flag=True
            break

possible initial state for p0, p5 = 182, 665


In [29]:
l=1000
sequence_x0 = gen_sequence(polynoms_int[0], 182, l)
sequence_x1 = gen_sequence(polynoms_int[1], 800, l)
sequence_x2 = gen_sequence(polynoms_int[2], 309, l)
sequence_x3 = gen_sequence(polynoms_int[3], 470, l)
sequence_x4 = gen_sequence(polynoms_int[4], 911, l)
sequence_x5 = gen_sequence(polynoms_int[5], 665, l)
seq = [combining_function(sequence_x0[k], sequence_x1[k], sequence_x2[k], sequence_x3[k], sequence_x4[k], sequence_x5[k]) for k in range(l)]
if seq == gamma[:l]:
    print("Success!           x0, x1, x2, x3, x4, x5 = " + str([182, 800, 309, 470, 911, 665]))

Success!           x0, x1, x2, x3, x4, x5 = [182, 800, 309, 470, 911, 665]
